In [ ]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 120.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:
from huggingface_hub import login
notebook_login()

In [ ]:
MODEL_NAME = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

(…)con-7b-instruct/resolve/main/config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

(…)uct/resolve/main/configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



(…)instruct/resolve/main/modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)ruct/resolve/main/generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

(…)truct/resolve/main/tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

(…)-7b-instruct/resolve/main/tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

(…)uct/resolve/main/special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


In [ ]:
prompt = """
<human>: what can i do to prevent poisoning by marine toxins?
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 2056
generation_config.temperature = 0.4
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: what can i do to prevent poisoning by marine toxins?
<assistant>: There are several things you can do to prevent poisoning by marine toxins. One of the most important things is to avoid consuming seafood that may be contaminated with toxins. It's also important to be aware of any local warnings or closures of beaches or fishing areas due to toxin-related concerns. Additionally, it's important to follow any guidelines or regulations set by local authorities regarding the handling and disposal of seafood.
User 
CPU times: user 11 s, sys: 373 ms, total: 11.4 s
Wall time: 11.3 s


In [ ]:
from datasets import load_dataset
train_data = load_dataset("keivalya/MedQuad-MedicalQnADataset")
train_data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/keivalya___csv/keivalya--MedQuad-MedicalQnADataset-9eb2b9d746e16e9a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['qtype', 'Question', 'Answer'],
        num_rows: 16407
    })
})

In [ ]:
train_data["train"][0]

{'qtype': 'susceptibility',
 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?',
 'Answer': 'LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.'}

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["Question"]}
<assistant>: {data_point["Answer"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

train_data = train_data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [ ]:
train_data

Dataset({
    features: ['qtype', 'Question', 'Answer', 'input_ids', 'attention_mask'],
    num_rows: 16407
})

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=4,
      gradient_accumulation_steps=4, #4
      num_train_epochs=6,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=500,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
      push_to_hub=True,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

Cloning https://huggingface.co/Tonic/experiments into local empty directory.


Download file pytorch_model.bin:   0%|          | 16.0k/4.20G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.81k/3.81k [00:00<?, ?B/s]

Download file runs/Oct25_22-29-12_05ba174c2960/events.out.tfevents.1698272958.05ba174c2960.313.0: 100%|#######…

Clean file training_args.bin:  26%|##6       | 1.00k/3.81k [00:00<?, ?B/s]

...

Clean file runs/Oct25_22-30-25_05ba174c2960/1698273029.936718/events.out.tfevents.1698273029.05ba174c2960.313.…

Clean file pytorch_model.bin:   0%|          | 1.00k/4.20G [00:00<?, ?B/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.332300
1000,1.174400
1500,1.139100
2000,1.164700
2500,1.113100
3000,1.112600
3500,1.060500
4000,1.030900
4500,0.955100
5000,0.936400


TrainOutput(global_step=6150, training_loss=1.0597990553941183, metrics={'train_runtime': 30766.4612, 'train_samples_per_second': 3.2, 'train_steps_per_second': 0.2, 'total_flos': 1.1252790565109983e+18, 'train_loss': 1.0597990553941183, 'epoch': 6.0})

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "Tonic/GaiaMiniMed"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)
trainer.push_to_hub(PEFT_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload file pytorch_model.bin:   0%|          | 1.00/4.20G [00:00<?, ?B/s]

Upload file runs/Oct25_23-51-09_efbe92b02989/events.out.tfevents.1698278905.efbe92b02989.1982.0:   0%|        …

To https://huggingface.co/Tonic/experiments
   8a9b6f6..bebfc3d  main -> main

   8a9b6f6..bebfc3d  main -> main

To https://huggingface.co/Tonic/experiments
   bebfc3d..fd5bf3c  main -> main

   bebfc3d..fd5bf3c  main -> main



'https://huggingface.co/Tonic/experiments/commit/bebfc3dc78c2c92b0b8a615c1f91426269f415e9'

In [ ]:
model.push_to_hub(
    PEFT_MODEL
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/Tonic/GaiaMiniMed/commit/6c0261bd112db44f985e7b06c73f552bf5de7e22', commit_message='Upload model', commit_description='', oid='6c0261bd112db44f985e7b06c73f552bf5de7e22', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

(…)edAware/resolve/main/adapter_config.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/12.6M [00:00<?, ?B/s]

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 2056
generation_config.temperature = 0.4
generation_config.top_p = 0.7
generation_config.num_return_sequences = 2
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: what can i do to prevent poisoning by marine toxins?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: what can i do to prevent poisoning by marine toxins?
<assistant>: You can help prevent poisoning by marine toxins by following these steps:
                
- Avoid eating shellfish.
                
- Avoid eating fish that are caught in the wild.
                
- Avoid eating fish that are caught in the wild.
                
- Avoid eating fish that are caught in the wild.
                
- Avoid eating fish that are caught in the wild.
                
- Avoid eating fish that are caught in the wild.
                

CPU times: user 14.9 s, sys: 70.2 ms, total: 14.9 s
Wall time: 15.4 s


In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: what are the symptoms of cancer?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: what are the symptoms of cancer?
<assistant>: Signs of cancer include a lump, a change in the size of the lump, or a change in the shape of the lump. A lump may be a symptom of cancer, but it is not always a sign of cancer. A lump may be a sign of a benign tumor, a tumor that is not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are not cancer. Benign tumors are
CPU times: user 14.8 s, sys: 34.4 ms, total: 14.8 s
Wall time: 15.1 s


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!zip -r /content/trained.zip /content/trained-model
!zip -r /content/experiments.zip /content/experiments

  adding: content/trained-model/ (stored 0%)
  adding: content/trained-model/adapter_config.json (deflated 42%)
  adding: content/trained-model/adapter_model.bin (deflated 7%)
  adding: content/experiments/ (stored 0%)
  adding: content/experiments/.gitattributes (deflated 87%)
  adding: content/experiments/checkpoint-6000/ (stored 0%)
  adding: content/experiments/checkpoint-6000/trainer_state.json (deflated 70%)
  adding: content/experiments/checkpoint-6000/scheduler.pt (deflated 48%)
  adding: content/experiments/checkpoint-6000/rng_state.pth (deflated 28%)
  adding: content/experiments/checkpoint-6000/training_args.bin (deflated 48%)
  adding: content/experiments/checkpoint-6000/optimizer.pt (deflated 23%)
  adding: content/experiments/checkpoint-6000/pytorch_model.bin (deflated 16%)
  adding: content/experiments/checkpoint-5500/ (stored 0%)
  adding: content/experiments/checkpoint-5500/trainer_state.json (deflated 69%)
  adding: content/experiments/checkpoint-5500/scheduler.pt (de

In [ ]:
from google.colab import files
files.download("/content/trained.zip")
files.download("/content/experiments.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): FalconForCausalLM(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (maybe_rotary): FalconRotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (dense): Linear4bit(in_